In [3]:
!git clone https://github.com/nachifur/RDDM.git

Cloning into 'RDDM'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 292 (delta 118), reused 105 (delta 40), pack-reused 74 (from 1)
Receiving objects: 100% (292/292), 9.22 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [7]:
import os 
os.chdir("/kaggle/working/RDDM/experiments/2_Image_Restoration_deraing_raindrop_noise1")

In [14]:
!pip install einops ema_pytorch Augmentor -q

In [15]:
import os
import sys

from src.denoising_diffusion_pytorch import GaussianDiffusion
from src.residual_denoising_diffusion_pytorch import (ResidualDiffusion,
                                                      Trainer, Unet, UnetRes,
                                                      set_seed)

# init 
os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(e) for e in [0])
sys.stdout.flush()
set_seed(10)
debug = False
if debug:
    save_and_sample_every = 2
    sampling_timesteps = 10
    sampling_timesteps_original_ddim_ddpm = 10
    train_num_steps = 200
else:
    save_and_sample_every = 1000
    sampling_timesteps = 5
    sampling_timesteps_original_ddim_ddpm = 250
    train_num_steps = 80000

original_ddim_ddpm = False
if original_ddim_ddpm:
    condition = False
    input_condition = False
    input_condition_mask = False
else:
    condition = True
    input_condition = True 
    input_condition_mask = True

if condition:
    if input_condition:
        folder = ["/home/sss/python/dataset/VOC2012/train/gt",
                "/home/sss/python/dataset/VOC2012/train/fs",
                "/home/sss/python/dataset/VOC2012/train/wSobel",
                "/home/sss/python/dataset/Celebrity Face Image Dataset/test/gt",
                "/home/sss/python/dataset/Celebrity Face Image Dataset/test/Dec",
                "/home/sss/python/dataset/Celebrity Face Image Dataset/test/sobel"]
    else:
        folder = ["/home/shenss/python/dataset/VOC2012/train/gt",
                "/home/shenss/python/dataset/VOC2012/train/input_ht_rgb",
                "/home/shenss/python/dataset/VOC2012/test/gt",
                "/home/shenss/python/dataset/VOC2012/test/input_ht_rgb"]
    train_batch_size = 1
    num_samples = 1
    sum_scale = 1
    image_size = 256
else:
    folder = '/home/shenss/python/dataset/CelebA/img_align_celeba'
    train_batch_size = 32
    num_samples = 25
    sum_scale = 1
    image_size = 32

if original_ddim_ddpm:
    model = Unet(
        dim = 64,
        dim_mults = (1, 2, 4, 8)
    )
    diffusion = GaussianDiffusion(
        model,
        image_size=image_size,
        timesteps=1000,           # number of steps
        sampling_timesteps=sampling_timesteps_original_ddim_ddpm,
        loss_type='l1',            # L1 or L2
    )
else:
    model = UnetRes(
        dim=64,
        dim_mults=(1, 2, 4, 8),
        share_encoder=0, #1 0 -1，分别对应共享编码器、两个独立的 U-Net 和一个独立的 U-Net。
        condition=condition,
        input_condition=input_condition
    )
    diffusion = ResidualDiffusion(
        model,
        image_size=image_size,
        timesteps=1000,           # number of steps
        # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
        sampling_timesteps=sampling_timesteps,
        objective='pred_res_noise',  # pred_res or pred_noise
        loss_type='l1',            # L1 or L2 or huber(SmoothL1)
        condition=condition,
        sum_scale = sum_scale,
        input_condition=input_condition,
        input_condition_mask=input_condition_mask
    )

trainer = Trainer(
    diffusion,
    folder,
    train_batch_size=train_batch_size,
    num_samples=num_samples,
    train_lr=8e-5,
    train_num_steps=train_num_steps,         # total training steps
    gradient_accumulate_every=2,    # gradient accumulation steps
    ema_decay=0.995,                # exponential moving average decay
    amp=False,                        # turn on mixed precision
    convert_image_to="RGB",
    condition=condition,
    save_and_sample_every=save_and_sample_every,
    equalizeHist=False,
    crop_patch=False,
    generation = False
)

ModuleNotFoundError: No module named 'datasets.get_dataset'